In [8]:
import sys
sys.path.append('../RecSysRep/')

In [9]:
import Basics.Load as ld

URM_all, _, _, _, _ = ld.getCOOs()
ICM_all = ld.getICMall()

In [10]:
import os

ofp = "../models_temp/Similarity_Hybrid/ALL/"


models_to_combine_best = {
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'ICM_all': {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'TopPop': {},
                            'PureSVD': {'num_factors': 30},
                            'UserKNN': {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'EASE_R': {'l2_norm': 2700},
}



In [17]:
def model_init(recommender, name, args):
    path_name = ofp + name + '_ALL'+ ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name + '_ALL')
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name + '_ALL')

In [18]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [19]:
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender

In [20]:
recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)
recommender2 = SLIM_BPR_Cython(URM_all)
recommender3 = RP3betaRecommender(URM_all)
recommender4 = ItemKNNCBFRecommender(URM_all, ICM_all)
recommender5 = IALSRecommender(URM_all)
recommender6 = P3alphaRecommender(URM_all)
recommender7 = UserKNNCFRecommender(URM_all)
recommender8 = PureSVDRecommender(URM_all)
recommender9 = TopPop(URM_all)
recommender10 = EASE_R_Recommender(URM_all)

model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
model_init(recommender2, 'SLIMBPR', models_to_combine_best['SLIMBPR'])
model_init(recommender3, 'RP3beta', models_to_combine_best['RP3beta'])
model_init(recommender4, 'ICM_all', models_to_combine_best['ICM_all'])
model_init(recommender5, 'IALS', models_to_combine_best['IALS'])
model_init(recommender6, 'P3alpha', models_to_combine_best['P3alpha'])
model_init(recommender7, 'UserKNN', models_to_combine_best['UserKNN'])
model_init(recommender8, 'PureSVD', models_to_combine_best['PureSVD'])
model_init(recommender9, 'TopPop', models_to_combine_best['TopPop'])
model_init(recommender10, 'EASE_R', models_to_combine_best['EASE_R'])

../models_temp/Similarity_Hybrid/ALL/SLIMER_ALL.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/ALL/SLIMER_ALL'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/ALL/SLIMBPR_ALL.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/ALL/SLIMBPR_ALL'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/ALL/RP3beta_ALL.zip
Model found!
RP3betaRecommender: Loading model from file '../models_temp/Similarity_Hybrid/ALL/RP3beta_ALL'
RP3betaRecommender: Loading complete
../models_temp/Similarity_Hybrid/ALL/ICM_all_ALL.zip
Model found!
ItemKNNCBFRecommender: Loading model from file '../models_temp/Similarity_Hybrid/ALL/ICM_all_ALL'
ItemKNNCBFRecommender: Loading complete
../models_temp/Similarity_Hybrid/ALL/IALS_ALL.zip
Model found!
IALSRecommender: Loading model from file '../models_temp/Similarity_Hybrid/ALL/IALS_ALL'
IALSRecommender: Loading complete


In [22]:
recommenderHybrid1 = ItemKNNScoresHybridTwoRecommender(URM_all, recommender4, recommender9)
recommenderHybrid1.fit(alpha = 0.65)

recommenderHybrid2 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid1, recommender3)
recommenderHybrid2.fit(alpha = 0.10)

recommenderHybrid3 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid2, recommender2)
recommenderHybrid3.fit(alpha = 0.20)

recommenderHybrid4 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid3, recommender7)
recommenderHybrid4.fit(alpha = 0.25)

recommenderHybrid5 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid4, recommender5)
recommenderHybrid5.fit(alpha = 0.9)

recommenderHybrid6 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid5, recommender10)
recommenderHybrid6.fit(alpha = 0.45)

recommenderHybrid7 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid6, recommender1)
recommenderHybrid7.fit(alpha = 0.5)

In [ ]:
recommenderHybrid1 = ItemKNNScoresHybridTwoRecommender(URM_all, recommender4, recommender9)
recommenderHybrid1.fit(alpha = 0.65)

recommenderHybrid2 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid1, recommender3)
recommenderHybrid2.fit(alpha = 0.10)

recommenderHybrid3 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid2, recommender2)
recommenderHybrid3.fit(alpha = 0.20)

recommenderHybrid4 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid3, recommender7)
recommenderHybrid4.fit(alpha = 0.25)

recommenderHybrid5 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid4, recommender5)
recommenderHybrid5.fit(alpha = 0.9)

recommenderHybrid6 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid5, recommender10)
recommenderHybrid6.fit(alpha = 0.45)

recommenderHybrid7 = ItemKNNScoresHybridTwoRecommender(URM_all, recommenderHybrid6, recommender1)
recommenderHybrid7.fit(alpha = 0.5)

In [23]:

recommender = recommenderHybrid7

import pandas as pd
at = 10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)



0
3899 5433 16365 16359 8465 640 6061 2243 9130 8635
1
3473 14400 940 16365 16012 640 14358 15119 2118 13032
2
6177 1551 17022 16365 17330 14229 2748 15119 14181 4607


../RecSysRep\Recommenders\KNN\ItemKNNScoresHybridMultipleRecommender.py:106: RuntimeWarning: invalid value encountered in true_divide
  l2_2_scores = item_weights_2 / l2_2
